# Computação Eficiente

Python é linguagem OO pensada como interface para sistemas de computação complexos. Assim, operações aparentemente simples escondem algoritmos sofisticados:

In [1]:
2+1

3

In [2]:
type(2)

int

In [ ]:
(2).__add__(1)

_Observação_: use dir() para descobrir os atributos e métodos disponíveis para um objeto.

In [3]:
', '.join(dir(2))

'__abs__, __add__, __and__, __bool__, __ceil__, __class__, __delattr__, __dir__, __divmod__, __doc__, __eq__, __float__, __floor__, __floordiv__, __format__, __ge__, __getattribute__, __getnewargs__, __gt__, __hash__, __index__, __init__, __init_subclass__, __int__, __invert__, __le__, __lshift__, __lt__, __mod__, __mul__, __ne__, __neg__, __new__, __or__, __pos__, __pow__, __radd__, __rand__, __rdivmod__, __reduce__, __reduce_ex__, __repr__, __rfloordiv__, __rlshift__, __rmod__, __rmul__, __ror__, __round__, __rpow__, __rrshift__, __rshift__, __rsub__, __rtruediv__, __rxor__, __setattr__, __sizeof__, __str__, __sub__, __subclasshook__, __truediv__, __trunc__, __xor__, bit_length, conjugate, denominator, from_bytes, imag, numerator, real, to_bytes'

In [4]:
import numpy as np

In [5]:
v1 = np.random.rand(100)
v2 = np.random.rand(100)
v3 = v1 + v2
print(v3)

[0.37866892 1.80889528 1.06126037 0.95811378 0.94607218 1.01988369
 1.16886419 1.056206   0.91336304 0.74199711 1.42974597 1.14002526
 1.35127008 1.03765124 0.52968828 1.0091012  0.47908933 1.38501747
 1.6226184  1.71804014 1.44990423 1.4487021  1.52981619 1.76287014
 1.24204919 1.46789536 1.54067192 1.540453   1.60475913 0.52392561
 1.20527275 1.29848599 0.55517594 0.63529466 1.10994703 1.9620272
 1.57978735 1.35046153 0.81636423 0.69972496 0.53386217 1.04386062
 1.39270319 1.87306196 0.72703069 1.13079153 0.75798407 1.46004403
 1.1385731  0.97165145 1.05253571 1.50695036 0.84620535 1.81212414
 0.70699385 1.74989291 0.69802095 1.10616538 1.53906618 0.898859
 1.14529088 1.00097779 0.53024893 0.85916366 0.24319757 1.05174207
 0.3026155  0.73856354 1.21034348 0.08994994 1.45936498 1.42276821
 1.46522753 0.52882567 0.83416354 0.5002088  0.70271941 1.19942092
 1.25970141 1.56125933 0.8475318  1.00066766 0.38807857 1.44768832
 0.87646316 0.90322906 1.29255391 1.26462876 0.52110728 0.8732694

In [8]:
m1 = np.random.rand(100, 15)
m2 = np.random.rand(15, 110)
m2

array([[0.01185655, 0.82795848, 0.87691497, ..., 0.38779663, 0.12197401,
        0.23725622],
       [0.75421333, 0.13613592, 0.68267485, ..., 0.89887596, 0.73017399,
        0.78987842],
       [0.66087955, 0.32620476, 0.93192481, ..., 0.63612865, 0.73565504,
        0.54495513],
       ...,
       [0.25723118, 0.8753168 , 0.33747325, ..., 0.26501072, 0.41309417,
        0.64553014],
       [0.33917695, 0.47248653, 0.09108859, ..., 0.1448845 , 0.41655333,
        0.31779103],
       [0.09204173, 0.96917521, 0.62597936, ..., 0.43552354, 0.34539051,
        0.87422145]])

In [9]:
def matmul(m1, m2):
    r = np.zeros((m1.shape[0], m2.shape[1]))
    for i in range(m1.shape[0]):
        for j in range(m2.shape[1]):
            for k in range(m2.shape[0]):
                r[i][j] += m1[i][k] * m2[k][j]
    return r

In [10]:
%timeit matmul(m1, m2)

171 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit np.matmul(m1, m2)

21.8 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
171000/21.4

7990.654205607477

Será que esta diferença de desempenho se deve ao mau uso do Python no primeiro exemplo? Esse ganho de eficiência talvez não exista ao se comparar com o código tivesse sido escrito em C e compilado de forma otimizada. Vamos ver?

Comparação de multiplicação de matrizes em Numpy, Python (C-Python) e C

<img src="images/TLhDi" alt="NumpyVersusC" style="width: 500px;"/>

Por que, na comparação acima, o Numpy foi tão rápido? Mais rápido mesmo que um código puro escrito em C? Basicamente, porque o código em numpy corresponde a versões otimizadas do melhor algotimo (prático) conhecido para o problema, escrito originalmente em Fortran para o BLAS. Abaixo, podemos ver a implementação do algoritmo de Strassen -- $O(n^{2.83})$, usado pelo numpy. Note o detalhe do _loop unrolling_ para acelerar o processamento em um hardware real.

```C

/*  -- translated by f2c (version 19940927).
   You must link the resulting object file with the libraries:
	-lf2c -lm   (in that order)
*/

#include "f2c.h"

doublereal sdot_(integer *n, real *sx, integer *incx, real *sy, integer *incy)
{


    /* System generated locals */
    integer i__1;
    real ret_val;

    /* Local variables */
    static integer i, m;
    static real stemp;
    static integer ix, iy, mp1;


/*     forms the dot product of two vectors.   
       uses unrolled loops for increments equal to one.   
       jack dongarra, linpack, 3/11/78.   
       modified 12/3/93, array(1) declarations changed to array(*)   


    
   Parameter adjustments   
       Function Body */
#define SY(I) sy[(I)-1]
#define SX(I) sx[(I)-1]


    stemp = 0.f;
    ret_val = 0.f;
    if (*n <= 0) {
	return ret_val;
    }
    if (*incx == 1 && *incy == 1) {
	goto L20;
    }

/*        code for unequal increments or equal increments   
            not equal to 1 */

    ix = 1;
    iy = 1;
    if (*incx < 0) {
	ix = (-(*n) + 1) * *incx + 1;
    }
    if (*incy < 0) {
	iy = (-(*n) + 1) * *incy + 1;
    }
    i__1 = *n;
    for (i = 1; i <= *n; ++i) {
	stemp += SX(ix) * SY(iy);
	ix += *incx;
	iy += *incy;
/* L10: */
    }
    ret_val = stemp;
    return ret_val;

/*        code for both increments equal to 1   


          clean-up loop */

L20:
    m = *n % 5;
    if (m == 0) {
	goto L40;
    }
    i__1 = m;
    for (i = 1; i <= m; ++i) {
	stemp += SX(i) * SY(i);
/* L30: */
    }
    if (*n < 5) {
	goto L60;
    }
L40:
    mp1 = m + 1;
    i__1 = *n;
    for (i = mp1; i <= *n; i += 5) {
	stemp = stemp + SX(i) * SY(i) + SX(i + 1) * SY(i + 1) + SX(i + 2) * 
		SY(i + 2) + SX(i + 3) * SY(i + 3) + SX(i + 4) * SY(i + 4);
/* L50: */
    }
L60:
    ret_val = stemp;
    return ret_val;
} /* sdot_ */

```

### O Tensorflow é o numpy para máquinas vetoriais

Contudo, usa uma abordagem estática. Ou seja, o grafo de computação é declarado primeiro e, então, compilado e executado.

In [14]:
import tensorflow as tf

E:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [16]:
t3 = tf.matmul(t1, t2)

In [17]:
t3

<tf.Tensor 'MatMul:0' shape=(100, 110) dtype=float32>

In [22]:
with tf.Session() as s:
    %timeit s.run(t3)

249 µs ± 4.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
def sigmoid(t):
    return 1.0 / (1.0 + tf.exp(-t))

In [24]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [25]:
t1s = sigmoid(t1)

In [26]:
t1s

<tf.Tensor 'truediv:0' shape=(100, 15) dtype=float32>

In [27]:
with tf.Session() as s:
    print(s.run(t1s))

[[0.6811364  0.5363544  0.6938013  ... 0.51871985 0.6798468  0.57355547]
 [0.70945364 0.62030077 0.5567146  ... 0.72628546 0.7257262  0.62635213]
 [0.71024084 0.51102895 0.71338314 ... 0.50232536 0.67425895 0.6711652 ]
 ...
 [0.50703585 0.51820153 0.58686787 ... 0.7114796  0.5619643  0.6120476 ]
 [0.6633063  0.6648792  0.5655095  ... 0.65587413 0.61645013 0.6796473 ]
 [0.68717635 0.7019232  0.71756274 ... 0.65609914 0.538653   0.612126  ]]


In [28]:
t1 = tf.placeholder(shape = (100, 15), dtype = tf.float32)
t2 = tf.placeholder(shape = (15, 110), dtype = tf.float32)

In [29]:
t3 = tf.matmul(t1, t2)

In [30]:
t3

<tf.Tensor 'MatMul_1:0' shape=(100, 110) dtype=float32>

In [ ]:
with tf.Session() as s:
    print(s.run(t3, feed_dict = {t1: m1, t2: m2}))
    m1 = 2 * m1
    m2 = 3 * m2
    print(s.run(t3, feed_dict = {t1: m1, t2: m2}))

O problema com essa abordagem é que ela ainda é relativamente baixo nível. Logo, não é produtiva em termos de prototipação. Assim, surge a necessidade de APIs de mais alto nível ainda, como o Keras.